### Softmax Regression

In [173]:
import torch
import torch.nn.functional as F

torch.manual_seed(1)

Low Level implementation of Cost function

In [174]:
z_data = [[1, 2, 3], [3, 5 ,1]]
z = torch.FloatTensor(z_data)

In [175]:
# 1 -> 0.09 2 -> 0.2447 3 -> 0.6652 : softmax done
hypothesis = F.softmax(z, dim = 1)
print(hypothesis)

hypothesis.sum()

tensor([[0.0900, 0.2447, 0.6652],
        [0.1173, 0.8668, 0.0159]])


tensor(2.0000)

In [176]:
z = torch.rand(3, 5, requires_grad=True)

In [177]:
# z size : (3, 5), dim = 1 -> row-wise softmax, dim =0 -> colun-wise softmax
hypothesis = F.softmax(z, dim = 1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [178]:
y = torch.randint(5, (3, )).long()
print(y)

tensor([0, 2, 1])


In [179]:
# create zero tensor with same size of hypothesis tensor : (3, 5)
y_one_hot = torch.zeros_like(hypothesis)
# give 1 entry to tensor y
# y.unsqueeze(1) : (3, ) to (3, 1) 
# scatter_(1, unsqeeuze, 1) : give entry 1 to row-wise
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [180]:
print(y.unsqueeze(1))

tensor([[0],
        [2],
        [1]])


In [181]:
print(y_one_hot)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])


In [182]:
# hypothesis = F.softmax(z, dim = 1)
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


High Level implementation of Cost function

In [183]:
# F.softmax() + torch.log() = F.log_softmax()
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

In [184]:
# hypothesis = F.softmax(z, dim = 1)
# cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
(y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [185]:
# or can use
# nll : negative log likelihood
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [186]:
# or can even use
# contains even the softmax function
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

Implementation of Actual Softmax Regression

In [194]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [195]:
# train data
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2,
           2,
           2,
           1,
           1,
           1,
           0,
           0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

Low level implementation

In [196]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [197]:
# y_train : 2, 1, 0 -> 3 classes
# (8 * 4)_x * (4 * 3)_W = (8 * 3)_y
y_one_hot = torch.zeros(8, 3)
y_one_hot = y_one_hot.scatter(1, y_train.unsqueeze(1), 1)

In [198]:
print(y_one_hot.shape)
print(y_one_hot)

torch.Size([8, 3])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


In [192]:
# (8 * 4)_x * (4 * 3)_W = (8 * 3)_y
W = torch.zeros((4, 3), requires_grad = True)
b = torch.zeros((1, 3), requires_grad = True)

optimizer = optim.SGD([W, b], lr = 0.1)

In [193]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost : {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost : 1.098612
Epoch  100/1000 Cost : 0.704199
Epoch  200/1000 Cost : 0.622999
Epoch  300/1000 Cost : 0.565717
Epoch  400/1000 Cost : 0.515291
Epoch  500/1000 Cost : 0.467662
Epoch  600/1000 Cost : 0.421278
Epoch  700/1000 Cost : 0.375402
Epoch  800/1000 Cost : 0.329766
Epoch  900/1000 Cost : 0.285072
Epoch 1000/1000 Cost : 0.248155


High level implementation

In [199]:
# Use F.cross_entropy()
W = torch.zeros((4,3), requires_grad = True)
b = torch.zeros((1,3), requires_grad = True)

optimizer = optim.SGD([W, b], lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # F.cross_entropy() already contains softmax : pre-compute xW + b
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost : {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost : 1.098612
Epoch  100/1000 Cost : 0.704199
Epoch  200/1000 Cost : 0.622999
Epoch  300/1000 Cost : 0.565717
Epoch  400/1000 Cost : 0.515292
Epoch  500/1000 Cost : 0.467662
Epoch  600/1000 Cost : 0.421278
Epoch  700/1000 Cost : 0.375402
Epoch  800/1000 Cost : 0.329766
Epoch  900/1000 Cost : 0.285073
Epoch 1000/1000 Cost : 0.248155


Implementation using nn.Module

In [330]:
torch.manual_seed(1)

In [329]:
model = nn.Linear(4,3)
print(model.state_dict())

optimizer = optim.SGD(model.parameters(), lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost : {:.6f}'.format(epoch, nb_epochs, cost.item()))

OrderedDict([('weight', tensor([[ 0.2576, -0.2207, -0.0969,  0.2347],
        [-0.4707,  0.2999, -0.1029,  0.2544],
        [ 0.0695, -0.0612,  0.1387,  0.0247]])), ('bias', tensor([ 0.1826, -0.1949, -0.0365]))])
Epoch    0/1000 Cost : 1.616785
Epoch  100/1000 Cost : 0.658891
Epoch  200/1000 Cost : 0.573443
Epoch  300/1000 Cost : 0.518151
Epoch  400/1000 Cost : 0.473265
Epoch  500/1000 Cost : 0.433516
Epoch  600/1000 Cost : 0.396563
Epoch  700/1000 Cost : 0.360914
Epoch  800/1000 Cost : 0.325392
Epoch  900/1000 Cost : 0.289178
Epoch 1000/1000 Cost : 0.254148


Implementation using class

In [ ]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)
        print(self.linear.state_dict())

    def forward(self, x):
        return self.linear(x)

In [332]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr = 0.1)

OrderedDict([('weight', tensor([[ 0.2576, -0.2207, -0.0969,  0.2347],
        [-0.4707,  0.2999, -0.1029,  0.2544],
        [ 0.0695, -0.0612,  0.1387,  0.0247]])), ('bias', tensor([ 0.1826, -0.1949, -0.0365]))])


In [333]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost : {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost : 1.616785
Epoch  100/1000 Cost : 0.658891
Epoch  200/1000 Cost : 0.573443
Epoch  300/1000 Cost : 0.518151
Epoch  400/1000 Cost : 0.473265
Epoch  500/1000 Cost : 0.433516
Epoch  600/1000 Cost : 0.396563
Epoch  700/1000 Cost : 0.360914
Epoch  800/1000 Cost : 0.325392
Epoch  900/1000 Cost : 0.289178
Epoch 1000/1000 Cost : 0.254148
